In [1]:
!wget https://raw.githubusercontent.com/hellohaptik/HINT3/master/dataset/v1/train/sofmattress_train.csv

--2021-04-04 14:02:04--  https://raw.githubusercontent.com/hellohaptik/HINT3/master/dataset/v1/train/sofmattress_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11866 (12K) [text/plain]
Saving to: ‘sofmattress_train.csv’

sofmattress_train.c 100%[===================>]  11.59K  --.-KB/s    in 0s      

2021-04-04 14:02:04 (132 MB/s) - ‘sofmattress_train.csv’ saved [11866/11866]



In [2]:
!pip install transformers > /dev/null

In [3]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import TFAutoModel
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report
import os

In [4]:
SEED = 42

os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [5]:
df = pd.read_csv('sofmattress_train.csv')
df.head()

,sentence,label
0,You guys provide EMI option?,EMI
1,Do you offer Zero Percent EMI payment options?,EMI
2,0% EMI.,EMI
3,EMI,EMI
4,I want in installment,EMI


In [6]:
seq_len = 512
num_samples = len(df)

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
df['sentence']=df['sentence'].str.lower()

In [9]:


X_train, X_valid, y_train, y_valid = train_test_split(df['sentence'], df['label'], stratify=df['label'], test_size=0.2, random_state = 2008)

In [10]:
y_train =y_train.reset_index()
y_valid = y_valid.reset_index()

In [11]:
y_train['is_train'] = 1

In [12]:
y_valid['is_train']= 0


In [13]:
y = pd.concat([y_train, y_valid])

In [14]:
y

,index,label,is_train
0,244,ABOUT_SOF_MATTRESS,1
1,210,MATTRESS_COST,1
2,84,WARRANTY,1
3,48,ORTHO_FEATURES,1
4,117,WHAT_SIZE_TO_ORDER,1
...,...,...,...
61,72,COMPARISON,0
62,208,MATTRESS_COST,0
63,218,MATTRESS_COST,0
64,3,EMI,0


In [15]:
label = pd.get_dummies(y, columns=['label'])
label_train = label[label.is_train == 1].drop(columns=['index','is_train'])
label_valid = label[label.is_train == 0].drop(columns=['index','is_train'])

In [16]:
label_indices = label.columns[2:]

In [17]:
X_train_ids = []
X_train_masks = []

X_valid_ids = []
X_valid_masks = []

for phrase in X_train:
    tokens = tokenizer.encode_plus(phrase,
                                   max_length = seq_len,
                                   truncation=True,
                                   padding='max_length',
                                   add_special_tokens=True,
                                   return_tensors='tf')
    X_train_ids.append( tokens['input_ids'][0] )
    X_train_masks.append( tokens['attention_mask'][0] )
    

In [18]:
for phrase in X_valid:
    tokens = tokenizer.encode_plus(phrase,
                                   max_length = seq_len,
                                   truncation=True,
                                   padding='max_length',
                                   add_special_tokens=True,
                                   return_tensors='tf')
    X_valid_ids.append( tokens['input_ids'][0] )
    X_valid_masks.append( tokens['attention_mask'][0] )

In [19]:
X_train_ids = np.array(X_train_ids)
X_train_masks = np.array(X_train_masks)
X_valid_ids = np.array(X_valid_ids)
X_valid_masks = np.array(X_valid_masks)

In [20]:
dataset_train = tf.data.Dataset.from_tensor_slices((X_train_ids, X_train_masks, label_train))
dataset_valid = tf.data.Dataset.from_tensor_slices((X_valid_ids, X_valid_masks, label_valid))


In [21]:
dataset_train.take(1)

<TakeDataset shapes: ((512,), (512,), (21,)), types: (tf.int32, tf.int32, tf.uint8)>

In [22]:
def map_func(X_ids, X_masks, label):
    return {'input_ids': X_ids, 'attention_mask':X_masks}, label

In [23]:
dataset_train = dataset_train.map(map_func).shuffle(1000).batch(8, drop_remainder=True)

In [24]:
dataset_valid = dataset_valid.map(map_func).shuffle(1000).batch(8, drop_remainder=True)

In [25]:
dataset_train.take(1)

<TakeDataset shapes: ({input_ids: (8, 512), attention_mask: (8, 512)}, (8, 21)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.uint8)>

In [26]:
#--------------

In [27]:
bert = TFAutoModel.from_pretrained('bert-base-uncased')
bert.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


In [28]:
_input_ids = tf.keras.layers.Input(shape=(seq_len,), name='input_ids', dtype='int32')
_input_masks = tf.keras.layers.Input(shape=(seq_len, ), name='attention_mask', dtype='int32')

embeddings = bert.bert(_input_ids, _input_masks)[1]

x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
drop = tf.keras.layers.Dropout(0.5)(x)
y = tf.keras.layers.Dense(df['label'].nunique(), activation='softmax')(drop)

In [29]:
model = tf.keras.Model(inputs= [_input_ids, _input_masks], outputs=y)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense (Dense)                   (None, 1024)         787456      bert[0][1]                   

In [30]:
mc = tf.keras.callbacks.ModelCheckpoint('./best_model.h5',verbose=1, save_best_only=True)
plat = tf.keras.callbacks.ReduceLROnPlateau(patience=2, verbose=1)
es = tf.keras.callbacks.EarlyStopping(verbose=1, patience=5)

callbacks = [mc, plat, es]

In [31]:
optimizer = tf.keras.optimizers.Adam(lr = 1e-4)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
model.compile(optimizer,loss = loss, metrics = [acc])

In [32]:
model.fit(dataset_train, epochs=25, validation_data=dataset_valid, callbacks=[plat,mc, es])

Epoch 1/25
32/32 [==============================] - 41s 1s/step - loss: 3.2329 - accuracy: 0.0653 - val_loss: 2.8898 - val_accuracy: 0.1250

Epoch 00001: val_loss improved from inf to 2.88983, saving model to ./best_model.h5
Epoch 2/25
32/32 [==============================] - 32s 993ms/step - loss: 2.9390 - accuracy: 0.0674 - val_loss: 2.2990 - val_accuracy: 0.3906

Epoch 00002: val_loss improved from 2.88983 to 2.29903, saving model to ./best_model.h5
Epoch 3/25
32/32 [==============================] - 33s 1s/step - loss: 2.3138 - accuracy: 0.3070 - val_loss: 1.4399 - val_accuracy: 0.5938

Epoch 00003: val_loss improved from 2.29903 to 1.43991, saving model to ./best_model.h5
Epoch 4/25
32/32 [==============================] - 32s 1s/step - loss: 1.5103 - accuracy: 0.5673 - val_loss: 1.1767 - val_accuracy: 0.7656

Epoch 00004: val_loss improved from 1.43991 to 1.17668, saving model to ./best_model.h5
Epoch 5/25
32/32 [==============================] - 33s 1s/step - loss: 0.7928 - accu

In [33]:
!wget https://raw.githubusercontent.com/hellohaptik/HINT3/master/dataset/v1/test/sofmattress_test.csv

--2021-04-04 14:14:02--  https://raw.githubusercontent.com/hellohaptik/HINT3/master/dataset/v1/test/sofmattress_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19125 (19K) [text/plain]
Saving to: ‘sofmattress_test.csv’

sofmattress_test.cs 100%[===================>]  18.68K  --.-KB/s    in 0s      

2021-04-04 14:14:02 (120 MB/s) - ‘sofmattress_test.csv’ saved [19125/19125]



In [34]:
df_test = pd.read_csv('sofmattress_test.csv')
df_test.head()

,sentence,label
0,There are only 2 models,NO_NODES_DETECTED
1,Single,NO_NODES_DETECTED
2,What's difference between ergo and ortho,COMPARISON
3,Return order,RETURN_EXCHANGE
4,Hai not recieved my product,DELAY_IN_DELIVERY


In [35]:

def ret_token(phrase):
    tokens = tokenizer.encode_plus(phrase,
                                   max_length = seq_len,
                                   truncation=True,
                                   padding='max_length',
                                   add_special_tokens=True,
                                   return_tensors='tf', return_token_type_ids=False )
    
    return {'input_ids':tf.cast(tokens['input_ids'], tf.float64), 'attention_mask':tf.cast(tokens['attention_mask'], tf.float64)}

In [36]:
my_model = tf.keras.models.load_model('best_model.h5')

In [37]:
_predicted_labels = []
for item in tqdm(df_test['sentence']):
    ret = ret_token(item)
    probs = my_model.predict(ret)
    pred_label = label_indices[np.argmax(probs)]
    pred_label = "_".join(pred_label.split('_')[1:])
    _predicted_labels.append(pred_label)

In [50]:
set(df['label'].unique()) - set(df_test['label'].unique())

{'WARRANTY'}

In [51]:
set(df_test['label'].unique()) - set(df['label'].unique()) 

{'NO_NODES_DETECTED'}

In [39]:
print(classification_report(df_test['label'], _predicted_labels))

                       precision    recall  f1-score   support

100_NIGHT_TRIAL_OFFER       0.33      0.40      0.36         5
   ABOUT_SOF_MATTRESS       0.07      0.50      0.12         2
         CANCEL_ORDER       1.00      0.67      0.80         6
        CHECK_PINCODE       0.48      0.91      0.62        22
                  COD       0.67      1.00      0.80         8
           COMPARISON       0.33      0.33      0.33        18
    DELAY_IN_DELIVERY       0.53      0.77      0.62        13
         DISTRIBUTORS       0.32      0.86      0.46         7
                  EMI       0.55      0.75      0.63        16
        ERGO_FEATURES       0.30      1.00      0.46         3
             LEAD_GEN       0.78      0.88      0.82        16
        MATTRESS_COST       0.62      0.38      0.47        21
    NO_NODES_DETECTED       0.00      0.00      0.00       166
               OFFERS       0.44      0.89      0.59         9
         ORDER_STATUS       0.42      0.62      0.50  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:

def get_inscope_accuracy(actual_node, pred_node):
    total = 0
    correct = 0
    for act, pred in zip(actual_node, pred_node):
        if act == 'NO_NODES_DETECTED':
            continue
        total += 1
        if act == pred:
            correct += 1
    return correct/total

In [41]:
get_inscope_accuracy(df_test['label'], _predicted_labels)

0.6147186147186147

################################################################################

In [ ]:
# def get_metrics(filepath, thresh):
#     df = pd.read_csv(filepath)
#     pred_node = []
#     for index, row in df.iterrows(): 
#         if row['predicted_node_score'] < thresh:
#             pred_node.append('NO_NODES_DETECTED')
#         else:
#             pred_node.append(row['predicted_node'])
#     actual_node = list(df['label'])
#     accuracy = accuracy_score(actual_node, pred_node)
#     overall_f1 = f1_score(actual_node, pred_node, labels=list(set(actual_node)), average='weighted')
#     inscope_accuracy = get_inscope_accuracy(actual_node, pred_node)
#     oos_recall = get_oos_recall(actual_node, pred_node)
#     mcc = matthews_corrcoef(actual_node, pred_node)
#     return accuracy, overall_f1, inscope_accuracy, oos_recall, mcc